In [1]:
save_figures = False

In [8]:
from auxiliary_code.twitter_api_request import request_tweets
import pandas as pd
import matplotlib.dates as mdates
from time import sleep, time
import pickle

### Obtaining the tweets

This process takes several hours. If one wants to obtain the tweet dataset,
it is adviced to run this process in batches (i.e. obtain all tweets for each year seperately) to minimize the chance of data loss

In [38]:
country='KE'
tweets_tot, places_tot, authors_tot = [], [], []


start = pd.to_datetime("2018-07-01T00Z")
day = pd.DateOffset(days=1)
hour = pd.DateOffset(hours=1)
for i in range(0, 1400):
    for j in range(0, 24):
        date = start + day * i + hour * j
        begin = date.strftime('%Y-%m-%dT%H') + ':00:00.000Z'
        date = start + day * i + hour * (j + 1)
        end = date.strftime('%Y-%m-%dT%H') + ':00:00.000Z'
        s = time()
        tweets, places, authors = request_tweets('has:geo', country, 'en', begin, end, verbose=True, max_runs=1, max_results=30)
        e = time()
        sleep(max(4-(e-s), 0))
        tweets_tot += tweets
        places_tot += places
        authors_tot += authors

2019-01-18T01:28:43.000Z
2019-01-18T02:35:27.000Z
2019-01-18T03:47:41.000Z
2019-01-18T04:54:35.000Z
2019-01-18T05:55:07.000Z
2019-01-18T06:56:05.000Z
2019-01-18T07:56:31.000Z
2019-01-18T08:54:52.000Z
2019-01-18T09:55:45.000Z
2019-01-18T10:55:18.000Z
2019-01-18T11:54:08.000Z
2019-01-18T12:53:13.000Z
2019-01-18T13:53:55.000Z
2019-01-18T14:54:32.000Z
2019-01-18T15:55:07.000Z
2019-01-18T16:51:45.000Z
2019-01-18T17:53:20.000Z
2019-01-18T18:51:32.000Z
2019-01-18T19:54:10.000Z
2019-01-18T20:48:25.000Z
2019-01-18T21:38:21.000Z
2019-01-18T22:36:44.000Z
2019-01-18T23:35:45.000Z
2019-01-19T00:00:51.000Z
2019-01-19T03:45:58.000Z
2019-01-19T04:50:51.000Z
2019-01-19T05:50:57.000Z
2019-01-19T06:52:27.000Z
2019-01-19T07:54:55.000Z
2019-01-19T08:55:25.000Z
2019-01-19T09:51:54.000Z
2019-01-19T10:52:40.000Z
2019-01-19T11:52:54.000Z
2019-01-19T12:51:15.000Z
2019-01-19T13:52:25.000Z
2019-01-19T14:53:39.000Z
2019-01-19T15:55:53.000Z
2019-01-19T16:55:47.000Z
2019-01-19T17:54:31.000Z
2019-01-19T18:54:26.000Z


In [ ]:
get_center = lambda bbox: [(bbox[0]+bbox[2])/2, (bbox[1] + bbox[3])/2]
place_df = []
for place in places_tot:
    place_df.append(get_center(place['geo']['bbox']) + [place['id']])

place_df = pd.DataFrame(place_df)
place_df.columns = ['long', 'lat', 'place_id']
place_df = place_df.drop_duplicates()

In [ ]:
author_df = []
for author in authors_tot:
            author_df.append([author['id'], author['description'], author['username'], author['name'], author['created_at']])

author_df = pd.DataFrame(author_df).drop_duplicates()
author_df.columns = ['author_id', 'description', 'username', 'name', 'created_at']
author_df['created_at'] = pd.to_datetime(author_df['created_at']).dt.tz_localize(None)

In [1]:
tweet_df = []
for tweet in tweets_tot:
    if 'geo' not in tweet:
        tweet['geo'] = {'place_id': None}
    tweet_df.append([tweet['text'], tweet['author_id'], tweet['geo']['place_id'], tweet['public_metrics']['like_count'],  tweet['public_metrics']['reply_count'], tweet['created_at'], tweet['id'], tweet['source']])

cols = ['text', 'author_id', 'place_id', 'likes', 'replies', 'date', 'id', 'source']

tweet_df = pd.DataFrame(tweet_df, columns=cols)
tweet_df = tweet_df.merge(place_df, left_on='place_id', right_on='place_id').drop_duplicates()
tweet_df = tweet_df.merge(author_df, left_on='author_id', right_on='author_id').drop_duplicates()

tweet_df.head()


In [ ]:
tweet_df.to_csv('../datasets/kenya_tweets.csv')